##### EDA

In [2]:
import torchxrayvision as xrv

C:\Program Files\Python312\Lib\site-packages\torchxrayvision\utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
d = xrv.datasets.COVID19_Dataset(imgpath="data/images/", csvpath="data/csv/metadata.csv")

In [4]:
img = d[0]['img'][0]
print(f"sample image of shape {img.shape}")
print(f"dataset of length {len(d)}")
print(img)

sample image of shape (888, 882)
dataset of length 535
[[1024.      1015.9686  1007.93726 ... -469.83527 -477.86664 -477.86664]
 [1024.      1024.      1015.9686  ... -493.92938 -493.92938 -493.92938]
 [1024.      1024.      1015.9686  ... -501.96075 -509.99213 -509.99213]
 ...
 [1024.      1024.       983.84314 ... -172.6745  -156.61176 -148.58038]
 [1024.      1024.       983.84314 ... -252.98822 -244.95685 -244.95685]
 [1024.      1024.       983.84314 ... -437.70978 -429.6784  -421.64703]]


##### SummaryWriter

In [1]:
from torch.utils.tensorboard import SummaryWriter
import torchvision
writer1 = SummaryWriter("logs/COVID19_train")
writer2 = SummaryWriter("logs/COVID19_test")

##### Dataloader

In [2]:
from dataLoader import *
from torch.utils.data import DataLoader

C:\Users\Hoo Kai Sng\OneDrive - National University of Singapore\Documents\NUS\CS3244\Project\xrayenv\Lib\site-packages\torchxrayvision\utils.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
# train_dataset = Covid19DataSet('train', transform = 'vanilla')
augmented_dataset = Covid19DataSet('train', transform = 'augment')
validation_dataset = Covid19DataSet('val', transform = 'vanilla')
test_dataset = Covid19DataSet('test', transform = 'vanilla')

In [4]:
BATCH_SIZE = 8
train_loader = DataLoader(dataset = augmented_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4)
validation_loader = DataLoader(dataset = validation_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 4)
test_loader = DataLoader(dataset = test_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 4)

In [5]:
# from PIL import Image
# import torchvision.transforms as transforms
# d = xrv.datasets.COVID19_Dataset(imgpath = "data/images/", csvpath = "data/csv/metadata.csv")
# img = d[0]['img'][0]
# img = img + 1024
# img = np.divide(img, 2048)
# pil = Image.fromarray(img)
# ld = transforms.ToTensor()
# tensor = ld(pil)
# compose = transforms.Compose([
#     transforms.Resize((224,224)),
#     transforms.RandomHorizontalFlip(p = 1),
#     transforms.ToTensor(), 
# ])
# tensor2 = compose(pil)
# print(tensor2)

##### Building of model

In [6]:
from models import *
from torchvision import models
from torchinfo import summary  # or use torchsummary

In [7]:
model = models.resnet18(weights = 'DEFAULT')
with torch.no_grad():
    model.conv1.weight = nn.Parameter(model.conv1.weight.mean(dim=1, keepdim=True))
for params in model.parameters():
    params.requires_grad = False
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 1)

##### Training Model

In [9]:
from evaluation import *

In [10]:
resnet_trained = train_model(model, train_loader, validation_loader, writer1, writer2)

Training model
Epoch [1/5] |Training Loss: 4.6184 | Validation Loss: 15.6221
Epoch [2/5] |Training Loss: 4.8685 | Validation Loss: 15.9815
Epoch [3/5] |Training Loss: 3.7462 | Validation Loss: 12.6289
Epoch [4/5] |Training Loss: 1.7689 | Validation Loss: 6.0899
Epoch [5/5] |Training Loss: 2.6739 | Validation Loss: 8.6527
Training completed in 5mins 45.19s
Best Loss: 6.0899 | Best Accuracy: 2646%


##### Evaluation

In [11]:
recall, prec, t, f, n = get_metrics(resnet_trained, test_loader, writer1)
print(f"Total Entries: {n} | Positive Preds: {t} | Negative Preds: {f}")
print(f"Fine-tuned: Precision = {prec:.0f}% | Recall = {recall:.0f}%")

TypeError: cat() received an invalid combination of arguments - got (Tensor, dim=int), but expected one of:
 * (tuple of Tensors tensors, int dim = 0, *, Tensor out = None)
 * (tuple of Tensors tensors, name dim, *, Tensor out = None)


##### Visualisation

In [ ]:
from explain import *

In [ ]:
# Model visualisation with GradCam
sample_pred = infer(resnet_trained, sample_img)

# Create heatmap using cam
heatmap = get_cam(resnet_trained, sample_pred, sample_img)

# Show image with heatmap overlay
overlay_cam(heatmap, img_numpy)